In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mne # reads edf format
import glob
import random
import re
import wfdb # waveform database library wfdb.rdann reads annotation of physiology annotated data in ECG, EEG etc


In [2]:
channel_labels2 = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3','P3-O1',
                  'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',
                  'FZ-CZ', 'CZ-PZ']

#### The international 10-20 systems defines placement of scalp electrodes.
Older system use 18 pairs or 18 channels
Newer ones may have 23,24 or 26 pairs.
For this project use 18 pairs or 18 channels.

Furthermore some of the files have errors, including those with electrodes
> ['PZ-P3', 'P3-P7', 'P7-P9',  'O2-O1',  'O1-O7'].

Decision made to use 18 channels instead.


In [3]:
# Each patient is assigned an integer from 0 to 24
# Retrieve assigned patient number from file

path = r'data\raw_data\chb-mit-scalp-eeg-database-1.0.0'
data_folders = sorted(glob.glob(os.path.join(path, '*[0-9]')))

def file_id(folder):
    return [ name[-2:] for name in [file.rsplit('\\',2)[-1] for file in folder]]
print("Case ID: ")
print(file_id(data_folders))

Case ID: 
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']


In [4]:
# seizure  marking is in name of edf file, not a separate file
# split files for training and testing
train_test_ratio = 0.8
random.seed(80)
train_folders = sorted(random.sample(data_folders, round(len(data_folders) * train_test_ratio)))
test_folders = sorted([ file for file in data_folders if file not in train_folders])
print(f"train_folders' IDs: {file_id(train_folders)}, contains {len(train_folders)} files")
print(f"test_folders' IDs: {file_id(test_folders)}, contains {len(test_folders)} files")


train_folders' IDs: ['01', '02', '04', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24'], contains 19 files
test_folders' IDs: ['03', '05', '06', '11', '21'], contains 5 files


In [5]:
# Retrieve edf files

train_files = [ file for folder in train_folders for file in glob.glob(folder+'/*.edf')]
test_files = [ file for folder in test_folders for file in glob.glob(folder+'/*.edf')]

print(f"Train_files contains {len(train_files)} files")
print(f"Test_files contains {len(test_files)} files")
print(f"Random example from train_files: {random.choice(train_files)}")


Train_files contains 523 files
Test_files contains 163 files
Random example from train_files: data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_16.edf


Raw files have duplicate files or files with duplicate names

In [6]:
# MEG and EEG library (mne) reads neurophysiological data
# dir(sample_edf)  # show all attributes and methods

sample_edf = mne.io.read_raw_edf(train_files[0], preload=False) #110-15 seconds
sample_edf.info

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2273869948.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  sample_edf = mne.io.read_raw_edf(train_files[0], preload=False) #110-15 seconds


<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 11:42:54 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
 subject_info: 1 item (dict)
>

Signal Extraction

Codes extensively modified from work by [Mashahiro Goton](https://www.kaggle.com/code/hemangjindal/chb-mit-eeg-dataset-my-notebook#CHB-MIT-eeg-dataset-seizure-detection-demo). [Online] [Accesssed on 20 October 2024].

In [7]:
'''
Reading EEG data, modified from cell 60 of notebook by Masahiro Gotoh  [Online]
[available at https://www.kaggle.com/code/hemangjindal/chb-mit-eeg-dataset-my-notebook#CHB-MIT-eeg-dataset-seizure-detection-demo]
'''

class EdfToNpy:
    # constants
    WINDOW_TIME = 10  # segment size in second
    STEP_TIME = 4     # Step size in second
    SEIZURE_PROPORTION = 0.01    # proportion of non seizure, data is imbalanced with less than 1% comprising seizure data, in term of total time from owner's documentations
    TO_MICROVOLTS = 1e6

    def __init__(self, folder, save_to):
        self.save_to = save_to
        self.folder = folder


    def read_edf(self):
        count=0 # samples with seizure plus samples without seizure * seizure proportion (down sampling to try to balance both class)
        window_size=0

        for file in self.folder:
            edf_data = mne.io.read_raw_edf(file, preload=False)
            edf_labels = edf_data.ch_names
            # check all channel labels appear in edf_labels
            # acceptss files which have all the
            if sum([any([0 if re.match(c, l) is None else 1 for l in edf_labels]) for c in channel_labels]) == len(channel_labels):
                sampling_freq = int(1/(edf_data.times[1]-edf_data.times[0]))
                window_size = sampling_freq * EdfToNpy.WINDOW_TIME
                window_stride = sampling_freq * EdfToNpy.STEP_TIME

                # identity EEG signal with seizure. 'Seizure' appended to file name, and marked at time point of seizure activity inside file.
                # has seizure marks seizure/non-seizure as (1/0) for every data point. Its size equals number of data points
                has_seizure = np.zeros((edf_data.n_times,))
                if os.path.exists(file + '.seizures'):
                    has_annotation = wfdb.rdann(file, 'seizures')
                    # has_annotation.sample e.g [10,20, 300,400] means marked seizure from sample 10-20 , and 300-400 . [start1,end1, start2,end2, ...]]
                    for idx in range(int(has_annotation.sample.size / 2)):
                        has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1

                # create has seizure index, and calculate proportion of signal which shows seizures
                # if a window segment contains some seizure in signal, that window segment is classified as having 'seizure'
                has_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_data.n_times - window_size) // window_stride)])

                ### size of samples with and without seizure after subsampling
                ### Data imbalance, normal EEG comprises 99% of data, multiplying by no seizure portion 0.01% allows down sampling of non seizure data
                # np.where returns tuple of array of indices , np.where()[0] returns array of indices
                noseizure_n_size = round(EdfToNpy.SEIZURE_PROPORTION * np.where(has_seizure_idx==0)[0].size)
                seizure_n_size = np.where(has_seizure_idx > 0)[0].size
                count = count + noseizure_n_size + seizure_n_size
            edf_data.close()

        return count, len(channel_labels), window_size


    def extract_edf(self, n_samples, n_channel_labels, window_size):
        signals_np = np.zeros((n_samples, n_channel_labels, window_size), dtype=np.float32)
        labels_np = np.zeros(n_samples, dtype=np.int32)

        # Read  edf, rename file channel to match names from channel labels list. When files have multiple channel names, only first is picked.
        for number, file in enumerate(self.folder):
            log = f"Reading file {number} "
            edf_data = mne.io.read_raw_edf(file, preload=False)

            # accept edf files that have the pre-requisite channels as defined in channel_labels
            n_label_match = sum([any([0 if re.match(ch, ch_name) is None else 1 for ch_name in edf_data.ch_names]) for ch in channel_labels])
            if n_label_match == len(channel_labels):
                # files may contain duplicate names
                dict_ch_name = {sorted([ch_name for ch_name in edf_data.ch_names if re.match(ch, ch_name) is not None])[0]: ch for ch in channel_labels}
                edf_data.rename_channels(dict_ch_name)

                # Retrieve EEG (in microvolts) ,  annotations
                has_seizure = np.zeros((edf_data.n_times,))
                signals_ = edf_data.get_data(picks=channel_labels) * EdfToNpy.TO_MICROVOLTS

                if  os.path.exists(file+'.seizures'):
                    log = log + "positive seizure"
                    has_annotation = wfdb.rdann(file, 'seizures')
                    for idx in range(int(has_annotation.sample.size / 2)):
                        has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1
                else:
                    log = log + "negative seizure"

                # create seizure index, and calculate proportion of signal which shows seizures
                sampling_freq = int(1/(edf_data.times[1]-edf_data.times[0]))
                window_size = sampling_freq * EdfToNpy.WINDOW_TIME

                window_stride = sampling_freq * EdfToNpy.STEP_TIME
                has_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_data.n_times - window_size) // window_stride)])

                # populate numpy array with EEG , annotation data
                noseizure_n_size = round(EdfToNpy.SEIZURE_PROPORTION * np.where(has_seizure_idx==0)[0].size)
                seizure_n_size = np.where(has_seizure_idx > 0)[0].size


                # non seizure data are by far larger than seizure data. To avoid overfitting, and bias, non seizure data is randomly subsampled. This prevents model
                # from being overwhelmed by large non seizure data
                count = 0
                temp_negative = random.sample(list(np.where(has_seizure_idx == 0)[0]), noseizure_n_size)
                for value in temp_negative:
                    start_index = value * window_stride
                    stop_index = value * window_stride + window_size
                    signals_np[count, :, :] = signals_[:, start_index:stop_index ]

                    labels_np[count] = 0
                    count = count + 1
                #seizure

                temp_positive = list(np.where(has_seizure_idx > 0)[0])
                for value in temp_positive:
                    start_index = value * window_stride
                    stop_index = value * window_stride + window_size
                    signals_np[count, :, :] = signals_[:, start_index: stop_index]
                    labels_np[count] = 1
                    count = count + 1

                #print(f"{noseizure_n_size+seizure_n_size} EEG signals added {seizure_n_size} with, {noseizure_n_size} without seizures")
            else:
                print(f"Unable to read {file}")

            # close resource
            edf_data.close()

            # save signal and label files
        np.save(self.save_to + '_signals', signals_np)
        np.save(self.save_to + '_labels', labels_np)


        def show_eeg(self, signals):
            # show a sample of extracted signals (the last one)

            vertical_width = 250
            fs = 256

            fig, ax = plt.subplots()
            for i in range(signals.shape[0]):
                ax.plot(np.arange(signals.shape[-1])/fs, signals[i, :]+i*vertical_width, linewidth=0.5, color='tab:blue')
                ax.annotate(channel_labels[i], xy=(0, i*vertical_width))
            ax.invert_yaxis()
            plt.show()


In [8]:
# Instantiate class
train_npy =  EdfToNpy(train_files, 'train_10sec')

In [12]:
# Get training samples
sample_length, channel_length, window_length = train_npy.read_edf()
train_npy.extract_edf(sample_length, channel_length, window_length)

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF param

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Tem

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_38.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_39.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_40.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_41.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF param

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Tem

Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_06.edf...
EDF file detected
Setting channel info

...

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeW

Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb20\chb20_34.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb20\chb20_59.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb20\chb20_59.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb20\chb20_60.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb20\chb20_60.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb20\chb20_68.edf...
EDF file detected
Setting channel info structure...
Creating raw.info stru

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '.'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '.'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '.'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\msp

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb23\chb23_17.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb23\chb23_19.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb23\chb23_19.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb23\chb23_20.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb23\chb23_20.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb24\chb24_01.edf...
EDF file detected
Sett

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Tem

In [10]:
# Instantiate class
test_npy = EdfToNpy(test_files, 'test_10sec')



In [11]:
# Get training samples
sample_length, channel_length, window_length = test_npy.read_edf()
test_npy.extract_edf(sample_length, channel_length, window_length)

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF param

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Tem

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_30.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_31.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_32.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb03\chb03_33.edf...
EDF file detected
Se

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:23: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Tem

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb05\chb05_20.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb05\chb05_21.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb05\chb05_22.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb05\chb05_23.edf...
EDF file detected
Se

...

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Tem

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_18.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_19.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_19.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_24.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_24.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_25.edf...
EDF file detected
Sett

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\msp


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb21\chb21_14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb21\chb21_14.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb21\chb21_15.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb21\chb21_15.edf
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb21\chb21_16.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Unable to read data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb21\chb21_16.edf
Extracting EDF parame

C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_14776\2612987023.py:63: RuntimeW

Numpy format
raw edf format parsed to  numpy format with each sample having 10-second window

In [24]:
# Training data
path_signal = 'train_10sec_signals.npy'
path_label = 'train_10sec_labels.npy'
train_signals = np.load(path_signal)
train_labels = np.load(path_label)

# Testing  data
path_signal = 'test_10sec_signals.npy'
path_label = 'test_10sec_labels.npy'
test_signals = np.load(path_signal)
test_labels = np.load(path_label)

In [25]:
train_signals.shape , train_labels.shape,  test_signals.shape, test_labels.shape

((9597, 18, 2560), (9597,), (2493, 18, 2560), (2493,))

In [11]:
n_samples, n_channels, segment_length = data_signals.shape
n_labels = data_labels.shape[0]

In [12]:
print(f"Numpy file {path_signal:<19} has {n_samples:<6} samples with {n_channels:} channels, and segment length of {segment_length}")
print(f"Numpy file {path_label:<19} has {n_labels:<6} labels ")

Numpy file train_10sec_signals.npy has 9597   samples with 18 channels, and segment length of 2560
Numpy file train_10sec_labels.npy has 9597   labels for seizure presence or absence.


In [13]:
unique, count = np.unique(train_labels, return_counts=True)
print(unique, count)

[0 1] [9478  119]


In [26]:
unique, count = np.unique(test_labels, return_counts=True)
print(unique, count)

[0 1] [2307  186]


In [ ]:
train_sing

In [ ]:
# END

In [ ]:

'''

# Configuration
csv_folder = '/kaggle/working/csv_files'  # Folder containing the merged CSV file
# eeg_channels = ['Channel_1', 'Channel_13', 'Channel_19', 'Channel_23']  # Channels to use
eeg_channels = ['Channel_5', 'Channel_15', 'Channel_3', 'Channel_2']  # Channels to use

window_size_sec = 5  # Window size in seconds
step_size_sec = 2 # Step size in seconds
sampling_rate = 16  # Sampling rate of EEG signals in Hz
preictal_label = 1
interictal_label = 0

# Derived parameters
window_size = window_size_sec * sampling_rate  # Convert to samples
step_size = step_size_sec * sampling_rate  # Convert to samples

# Function to load and prepare data
def load_and_prepare_data(csv_file):
    """
    Load the merged CSV data and prepare it for LSTM model training.
    """
    data = pd.read_csv(csv_file)

    # Normalize the EEG data (Min-Max scaling)
    scaler = MinMaxScaler()
    eeg_data = data[eeg_channels].values
    eeg_data_scaled = scaler.fit_transform(eeg_data)

    # Get the labels
    labels = data['Label'].values

    # Create sliding windows
    windows, window_labels = create_windows(eeg_data_scaled, labels, window_size, step_size)

    return windows, window_labels, scaler

# Function to create windows from the data
def create_windows(data, labels, window_size, step_size):
    """
    Create overlapping windows from EEG data and corresponding labels.
    """
    windows = []
    window_labels = []

    for start in range(0, len(data) - window_size + 1, step_size):
        end = start + window_size
        window = data[start:end, :]
        label = labels[start:end]

        # Assign label based on majority class in the window
        window_label = preictal_label if np.sum(label == preictal_label) > len(label) // 2 else interictal_label
        windows.append(window)
        window_labels.append(window_label)

    return np.array(windows), np.array(window_labels)

# Function to build the LSTM model
def build_lstm_model(input_shape):
    """
    Build and compile an LSTM model.
    """
    print('Input shape-',input_shape)
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid for binary classification
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train the LSTM model
def train_lstm_model(windows, window_labels):
    """
    Train the LSTM model using the provided windows and labels.
    """
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(windows, window_labels, test_size=0.2, random_state=42)

    # Build and train the model
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels

    # Print classification report
    print("Classification Report on Test Data:")
    print(classification_report(y_test, y_pred))

    # Return the trained model
    return model, scaler

# Real-time prediction function
def real_time_prediction(model, scaler, window_size, interval=30):
    """
    Simulate real-time predictions every `interval` seconds using the trained LSTM model.
    """
    print(f"Starting real-time prediction... (Interval: {interval}s)")

    # Simulate receiving new data (use the merged data as a placeholder)
    while True:
        # Simulate receiving data
        # Replace this with real-time data fetching process
        # For now, we assume the data is available in 'merged_df'
        # Here we can simulate a chunk of data for prediction
        simulated_data = np.random.rand(window_size, len(eeg_channels))  # Simulating new data

        # Normalize the new data
        scaled_data = scaler.transform(simulated_data)

        # Reshape data to match LSTM input shape
        X_input = scaled_data.reshape(1, window_size, len(eeg_channels))

        # Predict the probability of seizure (preictal)
        prediction = model.predict(X_input)
        print(f"Predicted probability of seizure: {prediction[0][0]:.4f}")

        # Sleep for the specified interval (simulate 30 seconds)
        time.sleep(interval)

# Main function
if __name__ == "__main__":
    # Load the merged data
    csv_file = '/kaggle/working/csv_files/merged_data.csv'
    windows, window_labels, scaler = load_and_prepare_data(csv_file)

    # Train the LSTM model and evaluate on the test data
    model, scaler = train_lstm_model(windows, window_labels)

    # Save the model for later use
    model.save('/kaggle/working/lstm_model.h5')
    print("Model saved to /kaggle/working/lstm_model.h5")

    # Start real-time prediction
    real_time_prediction(model, scaler, window_size, interval=30)

'''